In [1]:
import torch
import torch.utils.data.dataset as Dataset
import torch.utils.data.dataloader as DataLoader
import numpy as np

In [2]:
# 自定义数据集类要继承Dataset抽象类
class myDataset(Dataset.Dataset):       
    #初始化，定义数据内容和标签
    def __init__(self, Data, Label):
        self.Data = Data
        self.Label = Label
    #返回数据集大小
    def __len__(self):
        return len(self.Data)
    #得到数据内容和标签
    def __getitem__(self, index):
        data = torch.Tensor(self.Data[index])
        label = torch.Tensor(self.Label[index])
        return data, label

In [3]:
# 定义data与label并使用myDataset实例化数据集
data = np.asarray([[1,2],[3,4],[5,6],[7,8],[9,10]])
label = np.asarray([[1],[2],[3],[4],[5]])
dataset = myDataset(data,label)
print('data length: {}'.format(dataset.__len__()))
for i in range(dataset.__len__()):
    print(dataset.__getitem__(i))

data length: 5
(tensor([1., 2.]), tensor([1.]))
(tensor([3., 4.]), tensor([2.]))
(tensor([5., 6.]), tensor([3.]))
(tensor([7., 8.]), tensor([4.]))
(tensor([ 9., 10.]), tensor([5.]))


In [4]:
# 第一个参数为定义的dateset类的数据集，第二个参数batch_size为一个batch的大小
# 第三个参数shuffle表示是否打乱数据集，第四个参数num_workers表示使用几个线程来载入数据
dataloader = DataLoader.DataLoader(dataset,batch_size= 2, shuffle = False, num_workers= 2)

In [5]:
# 使用enumerate和for循环来遍历数据集，可以看到每个batch有两条数据，最后一个batch由于只剩一个数据则只有一条数据
for i, item in enumerate(dataloader):
        print('i:', i)
        data, label = item
        print('data:', data)
        print('label:', label)

i: 0
data: tensor([[1., 2.],
        [3., 4.]])
label: tensor([[1.],
        [2.]])
i: 1
data: tensor([[5., 6.],
        [7., 8.]])
label: tensor([[3.],
        [4.]])
i: 2
data: tensor([[ 9., 10.]])
label: tensor([[5.]])


In [6]:
# shuffle设置为true则数据集被打乱，每次运行结果不同
dataloader_with_shuffle = DataLoader.DataLoader(dataset,batch_size= 2, shuffle = True, num_workers= 2)
for i, item in enumerate(dataloader_with_shuffle):
        print('i:', i)
        data, label = item
        print('data:', data)
        print('label:', label)

i: 0
data: tensor([[7., 8.],
        [3., 4.]])
label: tensor([[4.],
        [2.]])
i: 1
data: tensor([[1., 2.],
        [5., 6.]])
label: tensor([[1.],
        [3.]])
i: 2
data: tensor([[ 9., 10.]])
label: tensor([[5.]])


In [7]:
# 定义数据集在gpu上的类
class myDataset_onGPU(Dataset.Dataset):       
    #初始化，定义数据内容和标签
    def __init__(self, Data, Label):
        self.Data = Data
        self.Label = Label
    #返回数据集大小
    def __len__(self):
        return len(self.Data)
    #得到数据内容和标签
    def __getitem__(self, index):
        data = torch.Tensor(self.Data[index])
        label = torch.Tensor(self.Label[index])
        if torch.cuda.is_available():
            data = data.cuda()
            label = label.cuda()
        return data, label

In [9]:
# 可以看到数据集已经在gpu cuda:0上
data = np.asarray([[1,2],[3,4],[5,6],[7,8],[9,10]])
label = np.asarray([[1],[2],[3],[4],[5]])
dataset_onGPU = myDataset_onGPU(data,label)
print('data length: {}'.format(dataset.__len__()))
for i in range(dataset_onGPU.__len__()):
    print(dataset_onGPU.__getitem__(i))

data length: 5
(tensor([1., 2.], device='cuda:0'), tensor([1.], device='cuda:0'))
(tensor([3., 4.], device='cuda:0'), tensor([2.], device='cuda:0'))
(tensor([5., 6.], device='cuda:0'), tensor([3.], device='cuda:0'))
(tensor([7., 8.], device='cuda:0'), tensor([4.], device='cuda:0'))
(tensor([ 9., 10.], device='cuda:0'), tensor([5.], device='cuda:0'))


In [10]:
# 如果num_workers不为0会报错”Cannot re-initialize CUDA in forked subprocess.“
# 这里设置num_workers = 0，只在主线程运行则没有问题
dataloader = DataLoader.DataLoader(dataset_onGPU,batch_size= 2, shuffle = False, num_workers= 0)
for i, item in enumerate(dataloader):
        print('i:', i)
        data, label = item
        print('data:', data)
        print('label:', label)

i: 0
data: tensor([[1., 2.],
        [3., 4.]], device='cuda:0')
label: tensor([[1.],
        [2.]], device='cuda:0')
i: 1
data: tensor([[5., 6.],
        [7., 8.]], device='cuda:0')
label: tensor([[3.],
        [4.]], device='cuda:0')
i: 2
data: tensor([[ 9., 10.]], device='cuda:0')
label: tensor([[5.]], device='cuda:0')


In [11]:
# 为了能使用多线程载入数据，因此推荐使用cpu载入数据后再将数据送入gpu
dataloader = DataLoader.DataLoader(dataset,batch_size= 2, shuffle = False, num_workers= 2)
for i, item in enumerate(dataloader):
        print('i:', i)
        data, label = item
        if torch.cuda.is_available():
            data = data.cuda()
            label = label.cuda()
        print('data:', data)
        print('label:', label)

i: 0
data: tensor([[1., 2.],
        [3., 4.]], device='cuda:0')
label: tensor([[1.],
        [2.]], device='cuda:0')
i: 1
data: tensor([[5., 6.],
        [7., 8.]], device='cuda:0')
label: tensor([[3.],
        [4.]], device='cuda:0')
i: 2
data: tensor([[ 9., 10.]], device='cuda:0')
label: tensor([[5.]], device='cuda:0')
